In [2]:
## A RouteTrie will store our routes and their associated handlers
class RouteTrie:
    def __init__(self, handler):
        # Initialize the trie with an root node and a handler, this is the root path or home page node
        self.root = RouteTrieNode()
        self.handler = handler

    def insert(self, path, handler=""):
        # Similar to our previous example you will want to recursively add nodes
        # Make sure you assign the handler to only the leaf (deepest) node of this path
        # path is a list of strings in between "/" in the full path
        
        current_node = self.root

        for next_string in path:            
            if next_string not in current_node.children:
                current_node.insert(next_string)
            current_node = current_node.children[next_string]

        # Set the handler at the last node
        current_node.handler = handler

    def find(self, path):
        # Starting at the root, navigate the Trie to find a match for this path
        # Return the handler for a match, or None for no match
        currentNode = self.root

        for next_string in path:
            try:
                currentNode = currentNode.children[next_string]
            except:
                return None
        return currentNode


## A RouteTrieNode will be similar to our autocomplete TrieNode... with one additional element, a handler.
class RouteTrieNode:
    def __init__(self):
        # Initialize the node with children as before, plus a handler
        self.children = {}
        self.handler = ""

    def insert(self, myString):
        # Insert the node as before
        self.children[myString] = RouteTrieNode()



## The Router class will wrap the Trie and handle 
class Router:
    def __init__(self, roothandler, notfoundhandler="404 page not found"):
        # Create a new RouteTrie for holding our routes
        # You could also add a handler for 404 page not found responses as well!
        self.handlerTrie = RouteTrie(roothandler)
        self.notfound_handler_Trie = RouteTrie(notfoundhandler)
        
        
    def add_handler(self, route, handler):
        # Add a handler for a path
        # You will need to split the path and pass the pass parts
        # as a list to the RouteTrie
        self.handlerTrie.insert(self.split_path(route), handler)

        
    def lookup(self, path):
        # lookup path (by parts) and return the associated handler
        # you can return None if it's not found or
        # return the "not found" handler if you added one
        # bonus points if a path works with and without a trailing slash
        # e.g. /about and /about/ both return the /about handler
        if path == "/":
            return(self.handlerTrie.handler)
        paths_list = self.split_path(path)
        
        returned_node = self.handlerTrie.find(paths_list)
        if not returned_node:
            # handler is None
            return(self.notfound_handler_Trie.handler)
            
        elif returned_node.handler == "":
            return(self.notfound_handler_Trie.handler)
            
        else:
            return(returned_node.handler)
        
               
    def split_path(self, path):
        # you need to split the path into parts for 
        # both the add_handler and loopup functions,
        # so it should be placed in a function here
        path_list = path.split("/")
        
        if path_list[0] == '':
            path_list = path_list[1:]
        if path_list[-1] == '':
            path_list = path_list[:-1]

        return path_list

In [6]:
## Here are some test cases and expected outputs you can use to test your implementation

## create the router and add a route
router = Router("root handler") # remove the 'not found handler' if you did not implement this
router.add_handler("/home/about", "about handler")  # add a route

## some lookups with the expected output
print(router.lookup("/")) # should print 'root handler'
print(router.lookup("/home")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/home/about")) # should print 'about handler'
print(router.lookup("/home/about/")) # should print 'about handler' or None if you did not handle trailing slashes
print(router.lookup("/home/about/me")) # should print 'not found handler' or None if you did not implement one

# My additional test cases
router.add_handler("/home2/about", "second handler")
print(router.lookup("/home2/about")) # Should print about handler
print("\n")
router.add_handler("/home", "home handler")
print(router.lookup("/")) # should print 'root handler'
print(router.lookup("/home")) # should print 'home handler' or None if you did not implement one
print(router.lookup("/home/about")) # should print 'about handler'


root handler
404 page not found
about handler
about handler
404 page not found
second handler


root handler
home handler
about handler
about handler
404 page not found
